In [1]:
import pandas as pd
import psycopg2

In [12]:
connection = psycopg2.connect("dbname=demand_prague user=postgres password=fidofido")
cursor = connection.cursor()

In [13]:
cursor.execute('SELECT version()')
cursor.fetchone()
cursor.close()

('PostgreSQL 13.2, compiled by Visual C++ build 1914, 64-bit',)

In [28]:
ids = "4465029, 4465030, 4465031, 4465032, 4465033, 4465034"

In [29]:
cursor = connection.cursor()
cursor.execute('SELECT * FROM sck_log.trip_log WHERE trip_id IN (4465029, 4465030, 4465031, 4465032, 4465033, 4465034)')
data = cursor.fetchall()
cursor.close()

In [30]:
columns = [
    "trip_id",
    "person_id",
    "from_activity_id",
    "to_activity_id",
    "start_time",
    "end_time",
    "duration",
    "transport_type"
]
trips = pd.DataFrame(data, columns = columns)

In [31]:
trips

,trip_id,person_id,from_activity_id,to_activity_id,start_time,end_time,duration,transport_type
0,4465029,360,0,1,30481000,31448000,967000,CAR
1,4465030,360,1,2,57120000,59432000,2312000,PT
2,4465031,800,0,1,38754000,39504000,750000,CAR
3,4465032,800,1,2,64305000,65055000,750000,CAR
4,4465033,327,0,1,30844000,37217000,6373000,BIKE
5,4465034,327,1,2,48182000,54104000,5922000,BIKE


In [33]:
# Activities
query = """
SELECT activity_log.*, node_log.* FROM sck_log.trip_log 
	LEFT JOIN sck_log.activity_log 
		ON (sck_log.trip_log.from_activity_id = sck_log.activity_log.activity_id OR sck_log.trip_log.to_activity_id = sck_log.activity_log.activity_id)
			AND sck_log.trip_log.person_id = sck_log.activity_log.person_id 
	LEFT JOIN sck_log.node_log
		ON sck_log.activity_log.location_id = sck_log.node_log.id 
	WHERE trip_id IN (4465029, 4465030, 4465031, 4465032, 4465033, 4465034)

""".format(ids)


cursor = connection.cursor()
cursor.execute(query)
data = cursor.fetchall()
cursor.close()
columns = [
    "person_id",
    "activity_id",
    "start_time",
    "end_time",
    "duration",
    "previous_trip_duration",
    "location_id",
    "attractor_id",
    "activity_type",
    "mode",
    "id",
    "latitude",
    "longitude",
    "info"
]
activities = pd.DataFrame(data, columns = columns)

In [34]:
activities

,person_id,activity_id,start_time,end_time,duration,previous_trip_duration,location_id,attractor_id,activity_type,mode,id,latitude,longitude,info
0,360,1,45094347,55174229,10079883,967000,LEISURE,127196,36774,CAR,127196,49.973589,14.074574,RoadNode
1,360,1,45094347,55174229,10079883,967000,LEISURE,127196,36774,CAR,127196,49.973589,14.074574,RoadNode
2,327,1,46322073,48182412,1860338,6373000,SHOP_DAILY,213352,1497,BIKE,213352,50.049700,14.295989,RoadNode
3,327,1,46322073,48182412,1860338,6373000,SHOP_DAILY,213352,1497,BIKE,213352,50.049700,14.295989,RoadNode
4,800,2,66605497,86400000,19794503,750000,SLEEP,228983,-1,CAR,228983,50.003349,13.996554,RoadNode
5,800,0,0,38754647,38754647,-1,SLEEP,228983,-1,NONE,228983,50.003349,13.996554,RoadNode
6,360,2,69786742,86400000,16613258,2312000,SLEEP,234365,-1,PT,234365,49.999599,14.001825,RoadNode
7,360,0,0,30481834,30481834,-1,SLEEP,234365,-1,NONE,234365,49.999599,14.001825,RoadNode
8,800,1,41054793,64305351,23250559,750000,LEISURE,251937,36773,CAR,251937,49.987220,14.067022,RoadNode
9,800,1,41054793,64305351,23250559,750000,LEISURE,251937,36773,CAR,251937,49.987220,14.067022,RoadNode
